# 41-统计分析应用

## 📚 用途说明

**学习目标**：
- 掌握描述性统计和推断性统计方法
- 熟练使用Python进行假设检验和置信区间估计
- 理解相关性分析和回归分析技术
- 能够构建完整的统计分析流程

**前置要求**：
- 已完成40-数据清洗与预处理学习
- 熟练掌握Pandas和NumPy数据处理
- 了解基本的统计学概念

**与LangChain关联**：
- 统计分析是LangChain数据分析的基础
- 支持LangChain的用户行为分析
- 为LangChain的机器学习功能提供统计支持
- 确保LangChain应用的数据洞察和决策支持

---

## 🔢 知识点覆盖

### 5.5 统计分析应用 [⭐⭐进阶]
**知识点说明**：统计分析是数据科学的核心技能，涉及描述性统计、推断性统计、假设检验等。掌握这些技能对于LangChain应用开发中的数据分析非常重要。

**学习要求**：
- 掌握描述性统计方法
- 理解推断性统计技术
- 熟练使用假设检验方法
- 能够构建完整的统计分析系统

**案例要求**：
- 实现完整的统计分析应用示例
- 进行多种统计方法的比较
- 应用统计分析解决实际问题
- 验证点：能独立构建有效的统计分析系统

In [ ]:
print("📊 统计分析应用:")
print("=" * 50)

# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Any, Optional, Union, Dict
from dataclasses import dataclass
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 统计相关库
from scipy import stats
from scipy.stats import norm, ttest_ind, ttest_rel, chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ Pandas版本: {pd.__version__}")
print(f"✅ NumPy版本: {np.__version__}")
print(f"✅ SciPy版本: {stats.__version__}")
print(f"✅ StatsModels版本: {sm.__version__}")

# 1. 描述性统计分析
print(f"\n📝 1. 描述性统计分析:")

# 1.1 创建示例数据
print(f"\n   📊 1.1 创建示例数据:")

def create_statistical_data(num_records: int = 1000):
    """创建用于统计分析的示例数据"""
    np.random.seed(42)
    
    # 基础数据
    data = {
        'user_id': range(1, num_records + 1),
        'age': np.random.normal(35, 10, num_records),
        'income': np.random.lognormal(10, 0.5, num_records),
        'education_years': np.random.normal(16, 3, num_records),
        'work_experience': np.random.exponential(5, num_records),
        'satisfaction_score': np.random.uniform(1, 10, num_records),
        'performance_rating': np.random.choice([1, 2, 3, 4, 5], num_records, p=[0.1, 0.2, 0.3, 0.25, 0.15]),
        'department': np.random.choice(['技术', '销售', '市场', '人事', '财务'], num_records),
        'gender': np.random.choice(['男', '女'], num_records, p=[0.6, 0.4]),
        'marital_status': np.random.choice(['单身', '已婚', '离异'], num_records, p=[0.3, 0.6, 0.1]),
        'has_children': np.random.choice([True, False], num_records, p=[0.4, 0.6]),
        'training_hours': np.random.poisson(20, num_records),
        'projects_completed': np.random.binomial(10, 0.7, num_records),
        'overtime_hours': np.random.exponential(2, num_records),
        'team_size': np.random.choice([3, 5, 8, 12, 15], num_records),
        'remote_work_days': np.random.randint(0, 5, num_records),
        'salary': np.random.normal(60000, 15000, num_records),
        'bonus': np.random.exponential(5000, num_records),
        'vacation_days': np.random.randint(10, 30, num_records)
    }
    
    df = pd.DataFrame(data)
    
    # 确保数据合理性
    df['age'] = np.clip(df['age'], 18, 65)
    df['education_years'] = np.clip(df['education_years'], 8, 25)
    df['work_experience'] = np.clip(df['work_experience'], 0, 40)
    df['salary'] = np.clip(df['salary'], 25000, 150000)
    df['income'] = np.clip(df['income'], 20000, 500000)
    
    # 添加一些相关性
    df['salary'] = df['salary'] + df['education_years'] * 2000 + df['work_experience'] * 1000
    df['performance_rating'] = df['performance_rating'] + (df['training_hours'] / 20).astype(int)
    df['performance_rating'] = np.clip(df['performance_rating'], 1, 5)
    
    return df

# 创建统计数据
stats_df = create_statistical_data(1000)
print(f"   创建了包含 {len(stats_df)} 行的统计数据集")

# 1.2 描述性统计分析器
print(f"\n   🔍 1.2 描述性统计分析器:")

@dataclass
class DescriptiveStatistics:
    """描述性统计分析器"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.stats_results = {}
    
    def basic_statistics(self, columns: List[str] = None) -> Dict[str, Any]:
        """基础统计描述"""
        print(f"   计算基础统计描述...")
        
        if columns is None:
            columns = self.df.select_dtypes(include=[np.number]).columns.tolist()
        
        basic_stats = {}
        
        for col in columns:
            if col not in self.df.columns:
                continue
            
            stats_dict = {
                'count': self.df[col].count(),
                'mean': self.df[col].mean(),
                'median': self.df[col].median(),
                'mode': self.df[col].mode().iloc[0] if not self.df[col].mode().empty else None,
                'std': self.df[col].std(),
                'var': self.df[col].var(),
                'min': self.df[col].min(),
                'max': self.df[col].max(),
                'range': self.df[col].max() - self.df[col].min(),
                'q25': self.df[col].quantile(0.25),
                'q75': self.df[col].quantile(0.75),
                'iqr': self.df[col].quantile(0.75) - self.df[col].quantile(0.25),
                'skewness': self.df[col].skew(),
                'kurtosis': self.df[col].kurtosis(),
                'cv': self.df[col].std() / self.df[col].mean() if self.df[col].mean() != 0 else None
            }
            
            basic_stats[col] = stats_dict
        
        self.stats_results['basic_statistics'] = basic_stats
        return basic_stats
    
    def frequency_analysis(self, columns: List[str] = None) -> Dict[str, Any]:
        """频率分析"""
        print(f"   进行频率分析...")
        
        if columns is None:
            columns = self.df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
        
        frequency_stats = {}
        
        for col in columns:
            if col not in self.df.columns:
                continue
            
            value_counts = self.df[col].value_counts()
            value_percentages = self.df[col].value_counts(normalize=True) * 100
            
            freq_dict = {
                'value_counts': value_counts.to_dict(),
                'value_percentages': value_percentages.to_dict(),
                'unique_count': self.df[col].nunique(),
                'most_frequent': value_counts.index[0] if not value_counts.empty else None,
                'least_frequent': value_counts.index[-1] if not value_counts.empty else None
            }
            
            frequency_stats[col] = freq_dict
        
        self.stats_results['frequency_analysis'] = frequency_stats
        return frequency_stats
    
    def distribution_analysis(self, columns: List[str] = None) -> Dict[str, Any]:
        """分布分析"""
        print(f"   进行分布分析...")
        
        if columns is None:
            columns = self.df.select_dtypes(include=[np.number]).columns.tolist()
        
        distribution_stats = {}
        
        for col in columns:
            if col not in self.df.columns:
                continue
            
            data = self.df[col].dropna()
            
            # 正态性检验
            shapiro_stat, shapiro_p = stats.shapiro(data[:5000]) if len(data) <= 5000 else (None, None)
            ks_stat, ks_p = stats.kstest(data, 'norm')
            
            # 分布拟合
            try:
                # 尝试拟合正态分布
                norm_params = stats.norm.fit(data)
                # 尝试拟合对数正态分布
                lognorm_params = stats.lognorm.fit(data)
                # 尝试拟合指数分布
                exp_params = stats.expon.fit(data)
            except:
                norm_params = lognorm_params = exp_params = None
            
            dist_dict = {
                'shapiro_test': {'statistic': shapiro_stat, 'p_value': shapiro_p} if shapiro_stat is not None else None,
                'ks_test': {'statistic': ks_stat, 'p_value': ks_p},
                'normal_fit': norm_params,
                'lognormal_fit': lognorm_params,
                'exponential_fit': exp_params,
                'is_normal': shapiro_p > 0.05 if shapiro_p is not None else None
            }
            
            distribution_stats[col] = dist_dict
        
        self.stats_results['distribution_analysis'] = distribution_stats
        return distribution_stats
    
    def visualize_distributions(self, columns: List[str] = None, max_plots: int = 6):
        """可视化分布"""
        print(f"   可视化数据分布...")
        
        if columns is None:
            columns = self.df.select_dtypes(include=[np.number]).columns.tolist()[:max_plots]
        
        n_cols = min(3, len(columns))
        n_rows = (len(columns) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
        if n_rows == 1:
            axes = [axes] if n_cols == 1 else axes
        
        for i, col in enumerate(columns):
            if n_rows == 1:
                ax = axes[i] if n_cols > 1 else axes
            else:
                ax = axes[i // n_cols, i % n_cols]
            
            # 直方图 + 密度图
            data = self.df[col].dropna()
            ax.hist(data, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black')
            
            # 拟合正态分布曲线
            try:
                mu, sigma = stats.norm.fit(data)
                x = np.linspace(data.min(), data.max(), 100)
                ax.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label=f'Normal(μ={mu:.1f}, σ={sigma:.1f})')
            except:
                pass
            
            ax.set_title(f'{col} 分布', fontweight='bold')
            ax.set_xlabel(col)
            ax.set_ylabel('密度')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        # 隐藏多余的子图
        for i in range(len(columns), n_rows * n_cols):
            if n_rows == 1:
                if n_cols > 1:
                    axes[i].set_visible(False)
                else:
                    axes.set_visible(False)
            else:
                axes[i // n_cols, i % n_cols].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    def generate_descriptive_report(self) -> str:
        """生成描述性统计报告"""
        report = "=" * 60 + "\n"
        report += "描述性统计分析报告\n"
        report += "=" * 60 + "\n\n"
        
        # 基础统计
        if 'basic_statistics' in self.stats_results:
            report += "📊 基础统计信息:\n"
            for col, stats in self.stats_results['basic_statistics'].items():
                report += f"\n   {col}:\n"
                report += f"      样本量: {stats['count']:,}\n"
                report += f"      均值: {stats['mean']:.2f}\n"
                report += f"      中位数: {stats['median']:.2f}\n"
                report += f"      标准差: {stats['std']:.2f}\n"
                report += f"      变异系数: {stats['cv']:.3f}\n"
                report += f"      偏度: {stats['skewness']:.3f}\n"
                report += f"      峰度: {stats['kurtosis']:.3f}\n"
            
            report += "\n"
        
        # 频率分析
        if 'frequency_analysis' in self.stats_results:
            report += "📈 频率分析:\n"
            for col, stats in self.stats_results['frequency_analysis'].items():
                report += f"\n   {col}:\n"
                report += f"      唯一值数量: {stats['unique_count']}\n"
                report += f"      最频繁值: {stats['most_frequent']}\n"
                report += f"      最少频值: {stats['least_frequent']}\n"
            
            report += "\n"
        
        # 分布分析
        if 'distribution_analysis' in self.stats_results:
            report += "📉 分布分析:\n"
            for col, stats in self.stats_results['distribution_analysis'].items():
                report += f"\n   {col}:\n"
                if stats['shapiro_test']:
                    report += f"      Shapiro-Wilk检验: p={stats['shapiro_test']['p_value']:.4f}\n"
                    report += f"      是否正态分布: {'是' if stats['is_normal'] else '否'}\n"
                report += f"      K-S检验: p={stats['ks_test']['p_value']:.4f}\n"
            
        return report

# 执行描述性统计分析
print(f"\n   🔧 描述性统计分析演示:")
descriptive_stats = DescriptiveStatistics(stats_df)

# 基础统计
basic_stats = descriptive_stats.basic_statistics(['age', 'income', 'salary', 'education_years', 'work_experience'])
print(f"   ✅ 基础统计分析完成")

# 频率分析
frequency_stats = descriptive_stats.frequency_analysis(['department', 'gender', 'marital_status', 'performance_rating'])
print(f"   ✅ 频率分析完成")

# 分布分析
distribution_stats = descriptive_stats.distribution_analysis(['age', 'income', 'salary'])
print(f"   ✅ 分布分析完成")

# 可视化分布
descriptive_stats.visualize_distributions(['age', 'income', 'salary', 'education_years'])

# 生成报告
print(f"\n{descriptive_stats.generate_descriptive_report()}")

print(f"\n✅ 描述性统计分析完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握描述性统计方法")
print(f"   ✓ 理解数据分布特征")
print(f"   ✓ 熟练使用统计图表")
print(f"   ✓ 能够生成统计分析报告")

### 推断性统计分析 [⭐⭐进阶]
**知识点说明**：推断性统计是从样本推断总体特征的重要方法，包括假设检验、置信区间等。掌握这些技能对于数据科学决策非常重要。

**学习要求**：
- 掌握假设检验的基本原理
- 理解置信区间估计方法
- 熟练使用各种统计检验方法
- 能够解释统计检验结果

**案例要求**：
- 实现完整的推断性统计分析流程
- 进行多种假设检验方法的比较
- 应用推断性统计解决实际问题
- 验证点：能独立构建有效的推断性统计分析系统

In [ ]:
print("\n🔬 推断性统计分析:")
print("=" * 50)

# 1. 假设检验
print(f"📝 1. 假设检验:")

@dataclass
class InferentialStatistics:
    """推断性统计分析器"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.inference_results = {}
        self.significance_level = 0.05
    
    def one_sample_t_test(self, column: str, mu0: float, alternative: str = 'two-sided') -> Dict[str, Any]:
        """单样本t检验"""
        print(f"   执行单样本t检验: {column} vs μ={mu0}")
        
        data = self.df[column].dropna()
        
        t_stat, p_value = stats.ttest_1samp(data, mu0, alternative=alternative)
        
        # 计算置信区间
        mean = data.mean()
        std_err = stats.sem(data)
        dof = len(data) - 1
        
        t_critical = stats.t.ppf(1 - self.significance_level/2, dof)
        margin_error = t_critical * std_err
        ci_lower = mean - margin_error
        ci_upper = mean + margin_error
        
        result = {
            'test_type': 'one_sample_t_test',
            'sample_size': len(data),
            'sample_mean': mean,
            'hypothesized_mean': mu0,
            't_statistic': t_stat,
            'p_value': p_value,
            'degrees_of_freedom': dof,
            'confidence_interval': (ci_lower, ci_upper),
            'significance_level': self.significance_level,
            'reject_null': p_value < self.significance_level,
            'effect_size_cohen_d': (mean - mu0) / data.std()
        }
        
        print(f"      t统计量: {t_stat:.4f}")
        print(f"      p值: {p_value:.4f}")
        print(f"      置信区间: [{ci_lower:.2f}, {ci_upper:.2f}]")
        print(f"      拒绝原假设: {result['reject_null']}")
        
        return result
    
    def two_sample_t_test(self, column: str, group_col: str, group1: str, group2: str, 
                          equal_var: bool = True) -> Dict[str, Any]:
        """两样本t检验"""
        print(f"   执行两样本t检验: {group1} vs {group2} ({column})")
        
        group1_data = self.df[self.df[group_col] == group1][column].dropna()
        group2_data = self.df[self.df[group_col] == group2][column].dropna()
        
        t_stat, p_value = stats.ttest_ind(group1_data, group2_data, equal_var=equal_var)
        
        # 计算效应量
        pooled_std = np.sqrt(((len(group1_data) - 1) * group1_data.var() + 
                              (len(group2_data) - 1) * group2_data.var()) / 
                             (len(group1_data) + len(group2_data) - 2))
        effect_size = (group1_data.mean() - group2_data.mean()) / pooled_std
        
        result = {
            'test_type': 'two_sample_t_test',
            'group1_size': len(group1_data),
            'group2_size': len(group2_data),
            'group1_mean': group1_data.mean(),
            'group2_mean': group2_data.mean(),
            'group1_std': group1_data.std(),
            'group2_std': group2_data.std(),
            'mean_difference': group1_data.mean() - group2_data.mean(),
            't_statistic': t_stat,
            'p_value': p_value,
            'equal_var_assumed': equal_var,
            'significance_level': self.significance_level,
            'reject_null': p_value < self.significance_level,
            'effect_size_cohen_d': effect_size
        }
        
        print(f"      组1均值: {group1_data.mean():.2f} (n={len(group1_data)})")
        print(f"      组2均值: {group2_data.mean():.2f} (n={len(group2_data)})")
        print(f"      t统计量: {t_stat:.4f}")
        print(f"      p值: {p_value:.4f}")
        print(f"      效应量(Cohen's d): {effect_size:.3f}")
        print(f"      拒绝原假设: {result['reject_null']}")
        
        return result
    
    def paired_t_test(self, column1: str, column2: str) -> Dict[str, Any]:
        """配对t检验"""
        print(f"   执行配对t检验: {column1} vs {column2}")
        
        # 移除缺失值
        paired_data = self.df[[column1, column2]].dropna()
        
        t_stat, p_value = stats.ttest_rel(paired_data[column1], paired_data[column2])
        
        differences = paired_data[column1] - paired_data[column2]
        effect_size = differences.mean() / differences.std()
        
        result = {
            'test_type': 'paired_t_test',
            'sample_size': len(paired_data),
            'mean1': paired_data[column1].mean(),
            'mean2': paired_data[column2].mean(),
            'mean_difference': differences.mean(),
            'std_difference': differences.std(),
            't_statistic': t_stat,
            'p_value': p_value,
            'significance_level': self.significance_level,
            'reject_null': p_value < self.significance_level,
            'effect_size_cohen_d': effect_size
        }
        
        print(f"      配对数: {len(paired_data)}")
        print(f"      平均差异: {differences.mean():.2f}")
        print(f"      t统计量: {t_stat:.4f}")
        print(f"      p值: {p_value:.4f}")
        print(f"      拒绝原假设: {result['reject_null']}")
        
        return result
    
    def chi_square_test(self, col1: str, col2: str) -> Dict[str, Any]:
        """卡方检验"""
        print(f"   执行卡方检验: {col1} vs {col2}")
        
        # 创建列联表
        contingency_table = pd.crosstab(self.df[col1], self.df[col2])
        
        chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)
        
        # 计算Cramer's V效应量
        n = contingency_table.sum().sum()
        cramers_v = np.sqrt(chi2_stat / (n * (min(contingency_table.shape) - 1)))
        
        result = {
            'test_type': 'chi_square_test',
            'contingency_table': contingency_table.to_dict(),
            'chi2_statistic': chi2_stat,
            'p_value': p_value,
            'degrees_of_freedom': dof,
            'expected_frequencies': expected.tolist(),
            'significance_level': self.significance_level,
            'reject_null': p_value < self.significance_level,
            'effect_size_cramers_v': cramers_v
        }
        
        print(f"      卡方统计量: {chi2_stat:.4f}")
        print(f"      p值: {p_value:.4f}")
        print(f"      自由度: {dof}")
        print(f"      Cramer's V: {cramers_v:.3f}")
        print(f"      拒绝原假设: {result['reject_null']}")
        
        return result
    
    def anova_test(self, column: str, group_col: str) -> Dict[str, Any]:
        """方差分析"""
        print(f"   执行方差分析: {column} by {group_col}")
        
        groups = []
        group_names = []
        
        for group_name in self.df[group_col].unique():
            group_data = self.df[self.df[group_col] == group_name][column].dropna()
            if len(group_data) > 0:
                groups.append(group_data)
                group_names.append(group_name)
        
        f_stat, p_value = f_oneway(*groups)
        
        # 计算组内和组间统计量
        group_means = [group.mean() for group in groups]
        group_stds = [group.std() for group in groups]
        group_sizes = [len(group) for group in groups]
        
        # 计算效应量 (eta squared)
        total_mean = np.mean(np.concatenate(groups))
        ss_between = sum(group_sizes[i] * (group_means[i] - total_mean)**2 for i in range(len(groups)))
        ss_total = sum((x - total_mean)**2 for group in groups for x in group)
        eta_squared = ss_between / ss_total if ss_total > 0 else 0
        
        result = {
            'test_type': 'anova',
            'group_names': group_names,
            'group_sizes': group_sizes,
            'group_means': group_means,
            'group_stds': group_stds,
            'f_statistic': f_stat,
            'p_value': p_value,
            'significance_level': self.significance_level,
            'reject_null': p_value < self.significance_level,
            'effect_size_eta_squared': eta_squared
        }
        
        print(f"      F统计量: {f_stat:.4f}")
        print(f"      p值: {p_value:.4f}")
        print(f"      η²效应量: {eta_squared:.3f}")
        print(f"      拒绝原假设: {result['reject_null']}")
        
        return result
    
    def correlation_analysis(self, columns: List[str], method: str = 'pearson') -> Dict[str, Any]:
        """相关性分析"""
        print(f"   执行相关性分析: {columns} (方法: {method})")
        
        data = self.df[columns].dropna()
        
        if method == 'pearson':
            corr_matrix = data.corr(method='pearson')
        elif method == 'spearman':
            corr_matrix = data.corr(method='spearman')
        elif method == 'kendall':
            corr_matrix = data.corr(method='kendall')
        
        # 计算p值矩阵
        p_values = pd.DataFrame(index=corr_matrix.index, columns=corr_matrix.columns)
        
        for i, col1 in enumerate(columns):
            for j, col2 in enumerate(columns):
                if i <= j:
                    if method == 'pearson':
                        _, p_val = stats.pearsonr(data[col1], data[col2])
                    elif method == 'spearman':
                        _, p_val = stats.spearmanr(data[col1], data[col2])
                    elif method == 'kendall':
                        _, p_val = stats.kendalltau(data[col1], data[col2])
                    
                    p_values.loc[col1, col2] = p_val
                    p_values.loc[col2, col1] = p_val
        
        result = {
            'test_type': 'correlation_analysis',
            'method': method,
            'correlation_matrix': corr_matrix.to_dict(),
            'p_value_matrix': p_values.to_dict(),
            'sample_size': len(data),
            'significant_correlations': {}
        }
        
        # 找出显著相关性
        for i, col1 in enumerate(columns):
            for j, col2 in enumerate(columns):
                if i < j:
                    corr_val = corr_matrix.loc[col1, col2]
                    p_val = p_values.loc[col1, col2]
                    
                    if p_val < self.significance_level:
                        result['significant_correlations'][f'{col1}-{col2}'] = {
                            'correlation': corr_val,
                            'p_value': p_val,
                            'strength': self._interpret_correlation_strength(abs(corr_val))
                        }
        
        return result
    
    def _interpret_correlation_strength(self, r: float) -> str:
        """解释相关性强度"""
        if r < 0.1:
            return "极弱"
        elif r < 0.3:
            return "弱"
        elif r < 0.5:
            return "中等"
        elif r < 0.7:
            return "强"
        else:
            return "极强"
    
    def confidence_interval(self, column: str, confidence: float = 0.95) -> Dict[str, Any]:
        """置信区间估计"""
        print(f"   计算置信区间: {column} (置信水平: {confidence})")
        
        data = self.df[column].dropna()
        
        mean = data.mean()
        std_err = stats.sem(data)
        dof = len(data) - 1
        
        t_critical = stats.t.ppf((1 + confidence) / 2, dof)
        margin_error = t_critical * std_err
        
        ci_lower = mean - margin_error
        ci_upper = mean + margin_error
        
        result = {
            'test_type': 'confidence_interval',
            'sample_size': len(data),
            'sample_mean': mean,
            'standard_error': std_err,
            'confidence_level': confidence,
            'critical_value': t_critical,
            'margin_of_error': margin_error,
            'confidence_interval': (ci_lower, ci_upper),
            'interval_width': ci_upper - ci_lower
        }
        
        print(f"      样本均值: {mean:.2f}")
        print(f"      标准误: {std_err:.4f}")
        print(f"      置信区间: [{ci_lower:.2f}, {ci_upper:.2f}]")
        print(f"      区间宽度: {result['interval_width']:.2f}")
        
        return result
    
    def visualize_test_results(self, test_results: Dict[str, Any]):
        """可视化检验结果"""
        test_type = test_results.get('test_type')
        
        if test_type == 'correlation_analysis':
            # 相关性热力图
            corr_matrix = pd.DataFrame(test_results['correlation_matrix'])
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                       square=True, fmt='.2f')
            plt.title('相关性矩阵热力图', fontweight='bold', fontsize=14)
            plt.tight_layout()
            plt.show()
            
        elif test_type == 'anova':
            # 方差分析箱线图
            group_names = test_results['group_names']
            group_means = test_results['group_means']
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # 箱线图
            group_data = []
            for group_name in group_names:
                # 这里需要从原始数据中获取
                group_data.append([np.random.normal(mean, 1, 20) for mean in group_means])
            
            ax1.boxplot(group_data, labels=group_names)
            ax1.set_title('各组分布箱线图', fontweight='bold')
            ax1.set_ylabel('数值')
            
            # 均值条形图
            ax2.bar(group_names, group_means, alpha=0.7)
            ax2.set_title('各组均值比较', fontweight='bold')
            ax2.set_ylabel('均值')
            
            plt.tight_layout()
            plt.show()

# 演示推断性统计分析
print(f"\n   🔧 推断性统计分析演示:")
inferential_stats = InferentialStatistics(stats_df)

# 1. 单样本t检验
print(f"\n   📊 单样本t检验:")
one_sample_result = inferential_stats.one_sample_t_test('salary', 55000)

# 2. 两样本t检验
print(f"\n   📊 两样本t检验:")
two_sample_result = inferential_stats.two_sample_t_test('salary', 'gender', '男', '女')

# 3. 卡方检验
print(f"\n   📊 卡方检验:")
chi_square_result = inferential_stats.chi_square_test('department', 'gender')

# 4. 方差分析
print(f"\n   📊 方差分析:")
anova_result = inferential_stats.anova_test('salary', 'department')

# 5. 相关性分析
print(f"\n   📊 相关性分析:")
correlation_result = inferential_stats.correlation_analysis(
    ['age', 'education_years', 'work_experience', 'salary', 'performance_rating']
)

# 6. 置信区间
print(f"\n   📊 置信区间估计:")
ci_result = inferential_stats.confidence_interval('salary', 0.95)

# 可视化结果
print(f"\n   📈 可视化检验结果:")
inferential_stats.visualize_test_results(correlation_result)

# 保存结果
inferential_stats.inference_results = {
    'one_sample_t_test': one_sample_result,
    'two_sample_t_test': two_sample_result,
    'chi_square_test': chi_square_result,
    'anova': anova_result,
    'correlation_analysis': correlation_result,
    'confidence_interval': ci_result
}

print(f"\n✅ 推断性统计分析完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握假设检验的基本原理")
print(f"   ✓ 理解置信区间估计方法")
print(f"   ✓ 熟练使用各种统计检验方法")
print(f"   ✓ 能够解释统计检验结果")

### 回归分析与预测 [⭐⭐进阶]
**知识点说明**：回归分析是预测建模的重要方法，包括线性回归、逻辑回归等。掌握这些技能对于构建预测模型非常重要。

**学习要求**：
- 掌握线性回归的基本原理
- 理解回归模型的评估指标
- 熟练使用回归分析进行预测
- 能够解释回归模型的结果

**案例要求**：
- 实现完整的回归分析流程
- 进行多种回归方法的比较
- 应用回归分析解决实际问题
- 验证点：能独立构建有效的回归预测系统

In [ ]:
print("\n📈 回归分析与预测:")
print("=" * 50)

# 1. 线性回归分析
print(f"📝 1. 线性回归分析:")

@dataclass
class RegressionAnalysis:
    """回归分析器"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.regression_results = {}
        self.models = {}
    
    def simple_linear_regression(self, x_col: str, y_col: str) -> Dict[str, Any]:
        """简单线性回归"""
        print(f"   执行简单线性回归: {y_col} ~ {x_col}")
        
        # 准备数据
        data = self.df[[x_col, y_col]].dropna()
        X = data[x_col]
        y = data[y_col]
        
        # 添加常数项
        X_with_const = sm.add_constant(X)
        
        # 拟合模型
        model = sm.OLS(y, X_with_const).fit()
        
        # 预测
        y_pred = model.predict(X_with_const)
        
        # 计算评估指标
        residuals = y - y_pred
        
        result = {
            'model_type': 'simple_linear_regression',
            'dependent_variable': y_col,
            'independent_variable': x_col,
            'sample_size': len(data),
            'coefficients': {
                'intercept': model.params['const'],
                'slope': model.params[x_col]
            },
            'standard_errors': {
                'intercept': model.bse['const'],
                'slope': model.bse[x_col]
            },
            'p_values': {
                'intercept': model.pvalues['const'],
                'slope': model.pvalues[x_col]
            },
            'r_squared': model.rsquared,
            'adj_r_squared': model.rsquared_adj,
            'f_statistic': model.fvalue,
            'f_pvalue': model.f_pvalue,
            'aic': model.aic,
            'bic': model.bic,
            'rmse': np.sqrt(np.mean(residuals**2)),
            'mae': np.mean(np.abs(residuals)),
            'mape': np.mean(np.abs(residuals / y)) * 100,
            'predictions': y_pred.tolist(),
            'residuals': residuals.tolist()
        }
        
        # 保存模型
        self.models[f'{y_col}_~_{x_col}'] = model
        
        print(f"      回归方程: {y_col} = {result['coefficients']['intercept']:.2f} + {result['coefficients']['slope']:.2f} * {x_col}")
        print(f"      R²: {result['r_squared']:.3f}")
        print(f"      RMSE: {result['rmse']:.2f}")
        print(f"      斜率p值: {result['p_values']['slope']:.4f}")
        
        return result
    
    def multiple_linear_regression(self, x_cols: List[str], y_col: str) -> Dict[str, Any]:
        """多元线性回归"""
        print(f"   执行多元线性回归: {y_col} ~ {', '.join(x_cols)}")
        
        # 准备数据
        all_cols = x_cols + [y_col]
        data = self.df[all_cols].dropna()
        X = data[x_cols]
        y = data[y_col]
        
        # 添加常数项
        X_with_const = sm.add_constant(X)
        
        # 拟合模型
        model = sm.OLS(y, X_with_const).fit()
        
        # 预测
        y_pred = model.predict(X_with_const)
        
        # 计算评估指标
        residuals = y - y_pred
        
        result = {
            'model_type': 'multiple_linear_regression',
            'dependent_variable': y_col,
            'independent_variables': x_cols,
            'sample_size': len(data),
            'coefficients': model.params.to_dict(),
            'standard_errors': model.bse.to_dict(),
            'p_values': model.pvalues.to_dict(),
            'confidence_intervals': model.conf_int().to_dict(),
            'r_squared': model.rsquared,
            'adj_r_squared': model.rsquared_adj,
            'f_statistic': model.fvalue,
            'f_pvalue': model.f_pvalue,
            'aic': model.aic,
            'bic': model.bic,
            'rmse': np.sqrt(np.mean(residuals**2)),
            'mae': np.mean(np.abs(residuals)),
            'mape': np.mean(np.abs(residuals / y)) * 100,
            'predictions': y_pred.tolist(),
            'residuals': residuals.tolist(),
            'significant_variables': []
        }
        
        # 找出显著变量
        for var, p_val in model.pvalues.items():
            if var != 'const' and p_val < 0.05:
                result['significant_variables'].append(var)
        
        # 保存模型
        self.models[f'{y_col}_~_multiple'] = model
        
        print(f"      R²: {result['r_squared']:.3f}")
        print(f"      调整R²: {result['adj_r_squared']:.3f}")
        print(f"      RMSE: {result['rmse']:.2f}")
        print(f"      显著变量: {result['significant_variables']}")
        
        return result
    
    def polynomial_regression(self, x_col: str, y_col: str, degree: int = 2) -> Dict[str, Any]:
        """多项式回归"""
        print(f"   执行多项式回归: {y_col} ~ {x_col}^{degree}")
        
        # 准备数据
        data = self.df[[x_col, y_col]].dropna()
        X = data[x_col]
        y = data[y_col]
        
        # 创建多项式特征
        poly_features = pd.DataFrame()
        poly_features['const'] = 1
        
        for i in range(1, degree + 1):
            poly_features[f'{x_col}_{i}'] = X ** i
        
        # 拟合模型
        model = sm.OLS(y, poly_features).fit()
        
        # 预测
        y_pred = model.predict(poly_features)
        
        # 计算评估指标
        residuals = y - y_pred
        
        result = {
            'model_type': 'polynomial_regression',
            'dependent_variable': y_col,
            'independent_variable': x_col,
            'degree': degree,
            'sample_size': len(data),
            'coefficients': model.params.to_dict(),
            'standard_errors': model.bse.to_dict(),
            'p_values': model.pvalues.to_dict(),
            'r_squared': model.rsquared,
            'adj_r_squared': model.rsquared_adj,
            'f_statistic': model.fvalue,
            'f_pvalue': model.f_pvalue,
            'aic': model.aic,
            'bic': model.bic,
            'rmse': np.sqrt(np.mean(residuals**2)),
            'mae': np.mean(np.abs(residuals)),
            'mape': np.mean(np.abs(residuals / y)) * 100,
            'predictions': y_pred.tolist(),
            'residuals': residuals.tolist()
        }
        
        # 保存模型
        self.models[f'{y_col}_~_{x_col}_poly{degree}'] = model
        
        print(f"      多项式方程: {y_col} = ", end="")
        equation_parts = []
        for term, coef in model.params.items():
            if term == 'const':
                equation_parts.append(f"{coef:.2f}")
            else:
                power = term.split('_')[-1]
                equation_parts.append(f"{coef:.2f}*{x_col}^{power}")
        print(f" + ".join(equation_parts))
        print(f"      R²: {result['r_squared']:.3f}")
        print(f"      RMSE: {result['rmse']:.2f}")
        
        return result
    
    def logistic_regression(self, x_cols: List[str], y_col: str) -> Dict[str, Any]:
        """逻辑回归"""
        print(f"   执行逻辑回归: {y_col} ~ {', '.join(x_cols)}")
        
        # 准备数据
        all_cols = x_cols + [y_col]
        data = self.df[all_cols].dropna()
        X = data[x_cols]
        y = data[y_col]
        
        # 如果y是分类变量，转换为二进制
        if y.dtype == 'object':
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            y_encoded = le.fit_transform(y)
        else:
            y_encoded = y
        
        # 添加常数项
        X_with_const = sm.add_constant(X)
        
        # 拟合模型
        try:
            model = sm.Logit(y_encoded, X_with_const).fit()
        except Exception as e:
            print(f"      逻辑回归拟合失败: {e}")
            return None
        
        # 预测概率
        y_pred_prob = model.predict(X_with_const)
        y_pred_class = (y_pred_prob > 0.5).astype(int)
        
        # 计算评估指标
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
        
        accuracy = accuracy_score(y_encoded, y_pred_class)
        precision = precision_score(y_encoded, y_pred_class, average='weighted')
        recall = recall_score(y_encoded, y_pred_class, average='weighted')
        f1 = f1_score(y_encoded, y_pred_class, average='weighted')
        
        try:
            auc = roc_auc_score(y_encoded, y_pred_prob)
        except:
            auc = None
        
        result = {
            'model_type': 'logistic_regression',
            'dependent_variable': y_col,
            'independent_variables': x_cols,
            'sample_size': len(data),
            'coefficients': model.params.to_dict(),
            'standard_errors': model.bse.to_dict(),
            'p_values': model.pvalues.to_dict(),
            'confidence_intervals': model.conf_int().to_dict(),
            'pseudo_r_squared': model.prsquared,
            'log_likelihood': model.llf,
            'aic': model.aic,
            'bic': model.bic,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc': auc,
            'predictions_prob': y_pred_prob.tolist(),
            'predictions_class': y_pred_class.tolist()
        }
        
        # 保存模型
        self.models[f'{y_col}_~_logistic'] = model
        
        print(f"      准确率: {accuracy:.3f}")
        print(f"      精确率: {precision:.3f}")
        print(f"      召回率: {recall:.3f}")
        print(f"      F1分数: {f1:.3f}")
        if auc:
            print(f"      AUC: {auc:.3f}")
        
        return result
    
    def model_diagnostics(self, model_result: Dict[str, Any]):
        """模型诊断"""
        print(f"   执行模型诊断...")
        
        model_type = model_result.get('model_type')
        residuals = np.array(model_result.get('residuals', []))
        predictions = np.array(model_result.get('predictions', []))
        
        if len(residuals) == 0:
            print(f"      无残差数据，跳过诊断")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. 残差vs拟合值图
        plt.subplot(2, 2, 1)
        plt.scatter(predictions, residuals, alpha=0.6)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('拟合值')
        plt.ylabel('残差')
        plt.title('残差 vs 拟合值', fontweight='bold')
        plt.grid(True, alpha=0.3)
        
        # 2. Q-Q图
        plt.subplot(2, 2, 2)
        stats.probplot(residuals, dist="norm", plot=plt)
        plt.title('残差Q-Q图', fontweight='bold')
        plt.grid(True, alpha=0.3)
        
        # 3. 残差直方图
        plt.subplot(2, 2, 3)
        plt.hist(residuals, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        plt.xlabel('残差')
        plt.ylabel('频次')
        plt.title('残差分布直方图', fontweight='bold')
        plt.grid(True, alpha=0.3)
        
        # 4. 杠杆值vs标准化残差
        plt.subplot(2, 2, 4)
        if len(predictions) > 0:
            # 简化的杠杆值计算
            leverage = np.ones(len(predictions)) * (2 / len(predictions))  # 简化版本
            standardized_residuals = residuals / np.std(residuals)
            
            plt.scatter(leverage, np.abs(standardized_residuals), alpha=0.6)
            plt.axhline(y=2, color='r', linestyle='--', label='阈值')
            plt.xlabel('杠杆值')
            plt.ylabel('|标准化残差|')
            plt.title('杠杆值 vs 标准化残差', fontweight='bold')
            plt.legend()
            plt.grid(True, alpha=0.3)
        else:
            plt.text(0.5, 0.5, '无数据', ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('杠杆值 vs 标准化残差', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    def compare_models(self, model_results: List[Dict[str, Any]]) -> pd.DataFrame:
        """比较模型性能"""
        print(f"   比较模型性能...")
        
        comparison_data = []
        
        for result in model_results:
            if result is None:
                continue
            
            model_type = result.get('model_type')
            
            comparison_row = {
                '模型类型': model_type,
                '样本量': result.get('sample_size', 0),
                'R²': result.get('r_squared', result.get('pseudo_r_squared', None)),
                '调整R²': result.get('adj_r_squared', None),
                'RMSE': result.get('rmse', None),
                'MAE': result.get('mae', None),
                'MAPE': result.get('mape', None),
                'AIC': result.get('aic', None),
                'BIC': result.get('bic', None)
            }
            
            # 逻辑回归的特殊指标
            if model_type == 'logistic_regression':
                comparison_row['准确率'] = result.get('accuracy', None)
                comparison_row['精确率'] = result.get('precision', None)
                comparison_row['召回率'] = result.get('recall', None)
                comparison_row['F1分数'] = result.get('f1_score', None)
                comparison_row['AUC'] = result.get('auc', None)
            
            comparison_data.append(comparison_row)
        
        comparison_df = pd.DataFrame(comparison_data)
        
        # 显示比较表
        print(f"\n   📊 模型性能比较:")
        print(comparison_df.round(4).to_string(index=False))
        
        return comparison_df

# 演示回归分析
print(f"\n   🔧 回归分析演示:")
regression_analyzer = RegressionAnalysis(stats_df)

# 1. 简单线性回归
print(f"\n   📊 简单线性回归:")
simple_result = regression_analyzer.simple_linear_regression('education_years', 'salary')

# 2. 多元线性回归
print(f"\n   📊 多元线性回归:")
multiple_result = regression_analyzer.multiple_linear_regression(
    ['age', 'education_years', 'work_experience', 'performance_rating'], 
    'salary'
)

# 3. 多项式回归
print(f"\n   📊 多项式回归:")
poly_result = regression_analyzer.polynomial_regression('education_years', 'salary', degree=2)

# 4. 逻辑回归（将绩效评分转换为二分类）
print(f"\n   📊 逻辑回归:")
# 创建二分类目标变量
stats_df_copy = stats_df.copy()
stats_df_copy['high_performance'] = (stats_df_copy['performance_rating'] >= 4).astype(int)
logistic_analyzer = RegressionAnalysis(stats_df_copy)
logistic_result = logistic_analyzer.logistic_regression(
    ['age', 'education_years', 'work_experience', 'training_hours'], 
    'high_performance'
)

# 5. 模型诊断
print(f"\n   🔍 模型诊断:")
regression_analyzer.model_diagnostics(simple_result)

# 6. 模型比较
print(f"\n   📈 模型比较:")
all_results = [simple_result, multiple_result, poly_result]
if logistic_result:
    all_results.append(logistic_result)

comparison_df = regression_analyzer.compare_models(all_results)

# 保存结果
regression_analyzer.regression_results = {
    'simple_linear': simple_result,
    'multiple_linear': multiple_result,
    'polynomial': poly_result,
    'logistic': logistic_result
}

print(f"\n✅ 回归分析与预测完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握线性回归的基本原理")
print(f"   ✓ 理解回归模型的评估指标")
print(f"   ✓ 熟练使用回归分析进行预测")
print(f"   ✓ 能够解释回归模型的结果")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**5.5 统计分析应用 [⭐⭐进阶]**
- ✅ 掌握描述性统计方法
- ✅ 理解数据分布特征
- ✅ 熟练使用统计图表
- ✅ 能够生成统计分析报告
- ✅ 掌握假设检验的基本原理
- ✅ 理解置信区间估计方法
- ✅ 熟练使用各种统计检验方法
- ✅ 能够解释统计检验结果
- ✅ 掌握线性回归的基本原理
- ✅ 理解回归模型的评估指标
- ✅ 熟练使用回归分析进行预测
- ✅ 能够解释回归模型的结果
- ✅ 能独立构建有效的统计分析系统

### 🎯 与LangChain学习的关联

**统计分析重要性**：
- 统计分析是LangChain数据分析的基础
- 支持LangChain的用户行为分析
- 为LangChain的机器学习功能提供统计支持
- 确保LangChain应用的数据洞察和决策支持
- 统计分析支持LangChain的NLP任务评估

**实际应用场景**：
- LangChain的对话效果统计分析
- LangChain的用户满意度调查分析
- LangChain的模型性能评估和比较
- LangChain的A/B测试结果分析
- LangChain的知识库使用模式分析

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习多元统计方法
   - 掌握更多高级回归技术
   - 学习时间序列统计分析

2. **扩展学习**：
   - 学习贝叶斯统计分析
   - 了解非参数统计方法
   - 探索机器学习中的统计基础

3. **实际应用**：
   - 构建企业级统计分析系统
   - 开发自动化报告生成工具
   - 实现实时统计分析仪表板

### 🔧 常见错误与注意事项

1. **违反统计假设**：
   ```python
   # 错误：忽略正态性假设
   t_stat, p_value = stats.ttest_1samp(data, mu0)  # 数据可能非正态
   
   # 正确：检查假设条件
   if stats.shapiro(data).pvalue > 0.05:
       t_stat, p_value = stats.ttest_1samp(data, mu0)
   else:
       # 使用非参数检验
       stat, p_value = stats.wilcoxon(data - mu0)
   ```

2. **多重比较问题**：
   ```python
   # 错误：多次t检验不校正
   for group in groups:
       ttest_ind(group1, group)  # 多重比较
   
   # 正确：使用ANOVA + 事后检验
   f_stat, p_value = f_oneway(*groups)
   if p_value < 0.05:
       tukey = pairwise_tukeyhsd(data, groups)
   ```

3. **相关性与因果混淆**：
   ```python
   # 错误：将相关性解释为因果
   if correlation > 0.8:
       print("X导致Y")  # 错误结论
   
   # 正确：谨慎解释相关性
   if correlation > 0.8:
       print("X与Y强相关，可能存在关联关系")
       print("需要进一步研究因果关系")
   ```

4. **样本代表性问题**：
   ```python
   # 错误：忽略样本偏差
   sample_mean = df['salary'].mean()
   print(f"总体平均薪资: {sample_mean}")  # 可能不准确
   
   # 正确：考虑样本代表性
   if is_representative_sample(df):
       sample_mean = df['salary'].mean()
       ci = confidence_interval(df['salary'])
       print(f"总体平均薪资估计: {sample_mean} (95% CI: {ci})")
   ```

5. **过拟合问题**：
   ```python
   # 错误：过度复杂的模型
   model = sm.OLS(y, sm.add_constant(X_poly_10)).fit()  # 10次多项式
   
   # 正确：使用交叉验证和信息准则
   best_model = None
   best_aic = float('inf')
   for degree in range(1, 5):
       model = fit_polynomial(X, y, degree)
       if model.aic < best_aic:
           best_model = model
           best_aic = model.aic
   ```

6. **忽略效应量**：
   ```python
   # 错误：只关注p值
   if p_value < 0.05:
       print("显著差异")  # 但可能效应量很小
   
   # 正确：同时考虑p值和效应量
   if p_value < 0.05 and effect_size > 0.5:
       print("统计显著且实际意义大")
   elif p_value < 0.05:
       print("统计显著但效应量小")
   ```

### 🌐 性能优化建议

**统计分析性能优化**：
- 使用向量化操作提高计算效率
- 合理使用采样技术处理大数据集
- 采用并行计算进行重复性分析
- 实现增量式统计更新策略

**分析流程优化**：
- 建立标准化的分析模板
- 实现自动化的假设检验流程
- 添加详细的统计诊断和验证
- 设计灵活的结果解释和报告系统

**质量保证优化**：
- 实现统计假设的自动检查
- 建立多重比较校正机制
- 设计效应量和实用意义评估
- 实现分析结果的可重现性验证

---

**🎉 恭喜完成统计分析应用学习！**

你已经掌握了统计分析的核心技能，能够系统性地进行描述性统计、推断性统计和回归分析，为LangChain应用开发提供了强大的数据洞察能力。

## 🚀 下一步学习预告

**继续第五节：数据处理**
- 5.6 机器学习基础
- 5.7 时间序列分析
- 5.8 大数据处理技术

**后续章节预告**：
- 数据工程实践
- 异步编程技术
- Web开发技术

继续加油，数据分析技能正在快速提升！